In [1]:
# 4

In [46]:
import numpy as np

In [47]:
data = np.array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362]) - 200

In [48]:
from scipy import stats

In [49]:
round(stats.wilcoxon(data, alternative="two-sided").pvalue, 4)

0.2845

In [22]:
# 5

In [50]:
data1 = np.array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
data2 = np.array([17, 18, 18, 15, 12, 4, 14, 15, 10])

In [53]:
round(stats.mannwhitneyu(data1, data2, alternative="greater").pvalue, 4)

0.029

In [54]:
# 6

In [55]:
import pandas as pd

In [88]:
data = pd.read_csv("challenger.txt", sep="\t")
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [89]:
data0 = data.Temperature[data.Incident == 0].values

In [90]:
data1 = data.Temperature[data.Incident == 1].values

In [91]:
np.random.seed(0)

In [92]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [93]:
mean_dif = get_bootstrap_samples(data0, 1000).mean(axis=1) - get_bootstrap_samples(data1, 1000).mean(axis=1)

In [94]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [96]:
rez = stat_intervals(mean_dif, 0.05)
rez

array([1.42299107, 7.93861607])

In [97]:
round(rez[0], 4)

1.423

In [98]:
# 7

In [99]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [104]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [101]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [102]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [105]:
np.random.seed(0)
permutation_test(data0, data1, max_permutations = 10000, alternative = 'two-sided')

0.007